In [1]:
"a"+"b"*238+"c"

'abbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbc'

In [ ]:
%%writefile watnu/config.stay

first_start: yes
database: watnu.sqlite
todo_files: [lesen.TODO]
throw: 1
lucky_num: 1
count: 6

In [ ]:
import datetime
import pony.orm as pny
 
database = pny.Database("sqlite",
                        "music.sqlite",
                        create_db=True)
 

class Artist(database.Entity):
    """
    Pony ORM model of the Artist table
    """
    name = pny.Required(unicode)
    albums = pny.Set("Album")
 

class Album(database.Entity):
    """
    Pony ORM model of album table
    """
    artist = pny.Required(Artist)
    title = pny.Required(unicode)
    release_date = pny.Required(datetime.date)
    publisher = pny.Required(unicode)
    media_type = pny.Required(unicode)
 
# turn on debug mode
pny.sql_debug(True)
 
# map the models to the database 
# and create the tables, if they don't exist
database.generate_mapping(create_tables=True)

In [ ]:
import datetime
import pony.orm as pny
 
from models import Album, Artist
 
#----------------------------------------------------------------------
@pny.db_session
def add_data():
    """"""
 
    new_artist = Artist(name=u"Newsboys")
    bands = [u"MXPX", u"Kutless", u"Thousand Foot Krutch"]
    for band in bands:
        artist = Artist(name=band)
 
    album = Album(artist=new_artist,
                  title=u"Read All About It",
                  release_date=datetime.date(1988,12,01),
                  publisher=u"Refuge",
                  media_type=u"CD")
 
    albums = [{"artist": new_artist,
               "title": "Hell is for Wimps",
               "release_date": datetime.date(1990,07,31),
               "publisher": "Sparrow",
               "media_type": "CD"
               },
              {"artist": new_artist,
               "title": "Love Liberty Disco", 
               "release_date": datetime.date(1999,11,16),
               "publisher": "Sparrow",
               "media_type": "CD"
              },
              {"artist": new_artist,
               "title": "Thrive",
               "release_date": datetime.date(2002,03,26),
               "publisher": "Sparrow",
               "media_type": "CD"}
              ]
 
    for album in albums:
        a = Album(**album)
 
if __name__ == "__main__":
    add_data()
 
    # use db_session as a context manager
    with pny.db_session:
        a = Artist(name="Skillet")

In [ ]:
from typing import List, Optional
from enum import Enum
from datetime import datetime
from pydantic.dataclasses import dataclass
from random import choice
l
ACTIVITY = Enum("ACTIVITY", "strength stamina intellect social spirit")
LEVEL = Enum("LEVEL", "MUST SHOULD MAY SHOULD_NOT MUST_NOT")

@dataclass
class Task:
    do: str
    namespace: Optional[str] = None
    #uuid: str = None
    
    comment: Optional[str] = None
    done: bool = False
    active: bool = True
    deleted: bool = False
    draft: bool = False
    
    priority: float = 1
    level: LEVEL = LEVEL.MAY
    
    activity: Optional[ACTIVITY] = None
    fear: float = 5
    difficulty: float = 5
    urgency: float = 5
    size: int = 1
    last_time: Optional[datetime] = None
    deadline: Optional[datetime] = None
    dependencies: Optional[List[str]] = None
    reconsider: Optional[str] = None
    ETC: Optional[int] = None
        

l = [Task(do="1", priority=1), Task(do="2", priority=2)]


def schedule_by_priority(L):
    def weight(t):
        return t.priority
    tasks = sorted(L, key=weight)
    yield tasks.pop()
    
def schedule_by_balance(L):
    pass

def schedule_by_timing(L):
    pass


In [ ]:
%%writefile Desktop/schedule.py

"""
Different ways to order tasks in a generalized Scheduler.

:Author: Anselm Kiefner
:Version: 2
:Date: 2018-11-12
"""
import random


def schedule_by_priority(tasks):
    '''Focus on priorities and getting stuff done.'''

    # since we have the tasks pre-ordered by level and priority, we can
    # assume this and only consider the items of the highest used level.
    # Actually testing this assertion is quite difficult, though.

    # we need to check for the two special cases first
    if len(tasks) == 0:
        return None

    if len(tasks) == 1:
        return tasks[0]

    considered_level = tasks[0].level
    considered_priority = tasks[0].priority

    # L is a list of the highest level and priority tasks only,
    # all tasks in this list are equal in both.

    L = []

    for t in tasks:
        # the first item is always in
        if t.level < considered_level:
            break
        if t.priority < considered_priority:
            break
        L.append(t)

    # Since all items in the list are equally viable, we make sure there
    # is no hidden arbitrary order.
    assert len(L) > 0, 'L must have at least one item, we checked that!'
    result = random.choice(L)

    return result

In [ ]:
def schedule_by_balance(tasks):
    '''Focus on balance of activities.

    1) Round-Robin of activities.
    2) Round-Robin of time spend on tasks.
    3) Round-Robin of poking at tasks (weakest criteria)
    '''

    # This needs to be done before passing the tasks in!
    # 1) find the activity A with the least time spent
    # 2) get all the tasks with A

    # Need to take care of the two degenerated cases first
    if len(tasks) == 0:
        return None
    if len(tasks) == 1:
        return tasks[0]

    def sort_by_time_spent_and_poke(L):
        assert all(t.activity == L[0].activity for t in L), \
            "Missed getting rid of tasks with ignored activities"

        # 3) poked at is the weaker criteria, so it gets first
        poked = sorted(L, key=lambda t: t.poked_at)
        # 4) time spend on tasks overrules poked at, so it gets second
        tended = sorted(poked, key=lambda t: t.get_abs_seconds())
        return tended

    assert len(tasks) > 0, 'How did we end up with an empty list?'
    result = sort_by_time_spent_and_poke(tasks)[0]

    return result

def schedule_by_timing(tasks):
    '''Schedule by timing. '''

    # 1) We check for tasks with the same tags as the last completed one
    # as we prefer related tasks as a baseline.
    # we remove task by task if they don't have a tag in common

    # TODO yeah.. thats obvious.
    if len(tasks) == 0:
        return None
    return tasks[0]

def orderMu(tasks):
    '''FIFO.'''
    try:
        return tasks[0]
    except KeyError:
        return None


def weight_task(task):
    return (0.3 * weight_by_affinity(task) +
            0.3 * weight_by_deadline(task) +
            0.3 * weight_by_ETC(task)
            )

def weight_by_affinity(task):
    """Take the calculated ."""
    return 0

def weight_by_deadline(task):
    return 0

def weight_by_ETC(task):
    return 0

def loop_the_list(order, tasks):
    """
    Scheduler functions only return the best fit per list, dropping the rest,
    which is normally the most efficient approach.
    For testing purposes however we need to see in which order a scheduler would
    return the items.
    To simplify testing, we take the order function and the tasks that need to
    be ordered, then we repeatedly feed the list, removing results one by one.
    We return the names of the tasks, concatenated as one string so we can test
    against 'abc', which greatly improves readability.
    We also have to make sure we can use it on queries.
    """

    tasks = list(tasks)

    result = ""

    for i in range(len(tasks)):
        feed = tasks.copy()
        X = order(feed, context)
        assert X in feed or X is None, "order isn't supposed to produce stuff.."
        try:
            result += X.name
            tasks.remove(X)
        # if order returns None
        except AttributeError:
            pass
    return result

To determine the optimal schedule (timer), there needs to be a model of the user including biological needs and customs.

In [ ]:
@dataclass
class Task:
    do: str
    namespace: str = field(keyword)
    uuid: UUID = field(default_factory(uuid4))

    comment: str = None
    done: bool = False
    active: bool = True
    deleted: bool = False
    draft: bool = False

    priority: float = 1
    level: LEVEL = LEVEL.MAY

    activity: ACTIVITY = None
    fear: float = 5
    difficulty: float = 5
    urgency: float = 5
    size: int = 1
    last_time: Optional[datetime] = None
    deadline: Optional[datetime] = None
    dependencies: Optional[List[str]] = None
    reconsider: Optional[str] = None
    ETC: Optional[int] = None

    def __hash__(self):
        return self.uuid.int

    def __str__(self):
        return self.do

In [ ]:
def schedule_by_random(tasks, *, count, coin, lucky_number):
    coin_throws = 0b1 + count
    seed((coin_throws ^ lucky_number) * count)
    return choice(tasks)

In [12]:
d = {"a": 1, "b": 2, "c":}

class Config:
    c = 1
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
        
c = Config(**d)

In [13]:
c.c

1

In [ ]:
class Config:
    def __init__(
        self,
        *,
        first_start: bool = True,
        database: str = "watnu.sqlite",
        throw: int = 0b1,
        lucky_num: int = 1,
        count: int = 1,
    ):
        self.__dict__.update(kwargs)

    def write(self):
        with open("config.stay", "w") as f:
            s = dumps(self)
            try:
                f.write(dumps(self))
            except:
                print("COULD NOT WRITE CONFIG!")

In [1]:
%cd /media/sf_Dropbox/code/watnu/master/src/watnu/

/media/sf_Dropbox/code/watnu/master/src/watnu


In [2]:
from PyQt5.QtSql import QSqlQuery, QSqlDatabase

In [3]:
db = QSqlDatabase.addDatabase("QSQLITE")
db.setDatabaseName("watnu.sqlite")
db.open()

True

In [4]:
query = QSqlQuery()

In [36]:
statement = f"""
CREATE TABLE IF NOT EXISTS spaces(
id INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE NOT NULL,
name VARCHAR(255) NOT NULL
)
"""

In [37]:
query.exec(statement)

True

In [42]:
    statement = f"""
    CREATE TABLE IF NOT EXISTS tasks (
    id INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE NOT NULL,
    do VARCHAR(280) NOT NULL,
    namespace_id INTEGER NOT NULL,
    done INTEGER NOT NULL DEFAULT 0,
    FOREIGN KEY (namespace_id) REFERENCES namespaces (id)
    )
    """

In [43]:
query.exec(statement)

True

In [4]:
from PyQt5.QtCore import QDateTime, QTime

In [6]:
time = QDateTime()
time.currentSecsSinceEpoch()

1608312636

In [18]:
cd ..

/media/sf_Dropbox/code/watnu


In [21]:
ll

insgesamt 32
drwxrwx--- 1 root 8192 Dez 17 22:10 box/
drwxrwx--- 1 root 4096 Dez 18 18:59 master/
drwxrwx--- 1 root 4096 Dez 11 17:34 proto-code-structure/
drwxrwx--- 1 root 4096 Jul 12 16:55 proto-loader/
drwxrwx--- 1 root 4096 Jul 11 15:08 proto-loader-core/
drwxrwx--- 1 root 4096 Dez  8 12:33 proto-sql/
drwxrwx--- 1 root 4096 Dez  3 15:29 proto-vertical/


In [22]:
from shlex import split

In [23]:
s = "#test foo bar #baz-bla #20"
parts = split(s)

['#test', 'foo', 'bar', '#baz-bla', '#20']

In [24]:
[p for p in split(s) if p.startswith("#")]

['#test', '#baz-bla', '#20']

In [28]:
cd master/src/watnu/lib

/media/sf_Dropbox/code/watnu/master/src/watnu/lib


In [29]:
from fluxx import StateMachine

In [30]:
from enum import Enum

In [31]:
S = Enum("STATE", "init main editing running final")


In [34]:
state = StateMachine(initial=S.init, 
    states={S.init:{S.main},
            S.main:{S.final, S.running, S.editing},
            S.running: {S.main},
            S.editing: {S.main, S.final}},
                    )

In [37]:
state() is S.init

True